In [102]:
from chi import server, context, lease
import os

context.version = "1.0" 
context.choose_project()
context.choose_site(default="CHI@UC")

In [103]:
# server.update_keypair()

In [104]:
l = lease.get_lease(f"model_serving_project32") 
l.show()

HTML(value='\n        <h2>Lease Details</h2>\n        <table>\n            <tr><th>Name</th><td>model_serving_…

Lease Details:
Name: model_serving_project32
ID: 49b9c1c0-4486-4753-9729-1212b1ce9b1a
Status: ACTIVE
Start Date: 2025-05-09 03:02:00
End Date: 2025-05-09 09:50:00
User ID: 9e387ebd353e6c7ce7bd7cf4cf30a7c19a9c20520f02131bb4abcc3f49788c11
Project ID: 7c0a7a1952e44c94aa75cae1ff5dc9b4

Node Reservations:
ID: a9a70e62-dd4d-4ba5-900b-6553c5e69475, Status: active, Min: 1, Max: 1

Floating IP Reservations:

Network Reservations:

Events:


In [105]:
username = os.getenv('USER') # all exp resources will have this prefix
s = server.Server(
    f"node-serve-model-{username}", 
    reservation_id=l.node_reservations[0]["id"],
    image_name="CC-Ubuntu24.04-CUDA"
)
s.submit(idempotent=True)

Waiting for server node-serve-model-nd2745_nyu_edu's status to become ACTIVE. This typically takes 10 minutes, but can take up to 20 minutes.


Server has moved to status ACTIVE


Attribute,node-serve-model-nd2745_nyu_edu
Id,50c7934d-c96d-4889-9907-ddfa4f51c507
Status,ACTIVE
Image Name,CC-Ubuntu24.04-CUDA
Flavor Name,baremetal
Addresses,sharednet1: IP: 10.140.82.189 (v4) Type: fixed MAC: 14:23:f2:a4:08:80
Network Name,sharednet1
Created At,2025-05-09T03:17:38Z
Keypair,nd2745_nyu_edu-jupyter
Reservation Id,a9a70e62-dd4d-4ba5-900b-6553c5e69475
Host Id,c15c5d0cd98629a41c320d11364f137f4320899eed52f609fb88500c


In [106]:
s.associate_floating_ip()

In [107]:
s.refresh()
s.check_connectivity()

Checking connectivity to 192.5.87.67 port 22.


Connection successful


In [108]:
s.refresh()
s.show(type="widget")

Attribute,node-serve-model-nd2745_nyu_edu
Id,50c7934d-c96d-4889-9907-ddfa4f51c507
Status,ACTIVE
Image Name,CC-Ubuntu24.04-CUDA
Flavor Name,baremetal
Addresses,sharednet1: IP: 10.140.82.189 (v4) Type: fixed MAC: 14:23:f2:a4:08:80 IP: 192.5.87.67 (v4) Type: floating MAC: 14:23:f2:a4:08:80
Network Name,sharednet1
Created At,2025-05-09T03:17:38Z
Keypair,nd2745_nyu_edu-jupyter
Reservation Id,a9a70e62-dd4d-4ba5-900b-6553c5e69475
Host Id,c15c5d0cd98629a41c320d11364f137f4320899eed52f609fb88500c


In [109]:
s.execute("curl -sSL https://get.docker.com/ | sudo sh")
s.execute("sudo groupadd -f docker; sudo usermod -aG docker $USER")
s.execute("docker run hello-world")

/opt/conda/lib/python3.10/site-packages/paramiko/client.py:889: UserWarning: Unknown ssh-ed25519 host key for 192.5.87.67: b'2c54e62007a8efba0bf3c08e962ce0d1'
  warnings.warn(


# Executing docker install script, commit: 53a22f61c0628e58e1d6680b49e82993d304b449


+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install ca-certificates curl >/dev/null
+ sh -c install -m 0755 -d /etc/apt/keyrings
+ sh -c curl -fsSL "https://download.docker.com/linux/ubuntu/gpg" -o /etc/apt/keyrings/docker.asc
+ sh -c chmod a+r /etc/apt/keyrings/docker.asc
+ sh -c echo "deb [arch=amd64 signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu noble stable" > /etc/apt/sources.list.d/docker.list
+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install docker-ce docker-ce-cli containerd.io docker-compose-plugin docker-ce-rootless-extras docker-buildx-plugin >/dev/null

Running kernel seems to be up-to-date.

The processor microcode seems to be up-to-date.

No services need to be restarted.

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.
+ sh -c doc

Client: Docker Engine - Community
 Version:           28.1.1
 API version:       1.49
 Go version:        go1.23.8
 Git commit:        4eba377
 Built:             Fri Apr 18 09:52:14 2025
 OS/Arch:           linux/amd64
 Context:           default

Server: Docker Engine - Community
 Engine:
  Version:          28.1.1
  API version:      1.49 (minimum version 1.24)
  Go version:       go1.23.8
  Git commit:       01f442b
  Built:            Fri Apr 18 09:52:14 2025
  OS/Arch:          linux/amd64
  Experimental:     false
 containerd:
  Version:          1.7.27
  GitCommit:        05044ec0a9a75232cad458027ca83437aae3f4da
 runc:
  Version:          1.2.5
  GitCommit:        v1.2.5-0-g59923ef
 docker-init:
  Version:          0.19.0
  GitCommit:        de40ad0


To run Docker as a non-privileged user, consider setting up the
Docker daemon in rootless mode for your user:

    dockerd-rootless-setuptool.sh install

Visit https://docs.docker.com/go/rootless/ to learn about rootless mode.


T

Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world
e6590344b1a5: Pulling fs layer
e6590344b1a5: Download complete
e6590344b1a5: Pull complete
Digest: sha256:c41088499908a59aae84b0a49c70e86f4731e588a737f1637e73c8c09d995654
Status: Downloaded newer image for hello-world:latest



Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/



<Result cmd='docker run hello-world' exited=0>

In [110]:
s.execute("curl -fsSL https://nvidia.github.io/libnvidia-container/gpgkey | sudo gpg --dearmor -o /usr/share/keyrings/nvidia-container-toolkit-keyring.gpg \
  && curl -s -L https://nvidia.github.io/libnvidia-container/stable/deb/nvidia-container-toolkit.list | \
    sed 's#deb https://#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://#g' | \
    sudo tee /etc/apt/sources.list.d/nvidia-container-toolkit.list")
s.execute("sudo apt update")
s.execute("sudo apt-get install -y nvidia-container-toolkit")
s.execute("sudo nvidia-ctk runtime configure --runtime=docker")
s.execute("sudo systemctl restart docker")


# s.execute("sudo jq 'if has(\"exec-opts\") then . else . + {\"exec-opts\": [\"native.cgroupdriver=cgroupfs\"]} end' /etc/docker/daemon.json | sudo tee /etc/docker/daemon.json.tmp > /dev/null && sudo mv /etc/docker/daemon.json.tmp /etc/docker/daemon.json")

deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://nvidia.github.io/libnvidia-container/stable/deb/$(ARCH) /
#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://nvidia.github.io/libnvidia-container/experimental/deb/$(ARCH) /


Get:1 https://nvidia.github.io/libnvidia-container/stable/deb/amd64  InRelease [1477 B]
Hit:2 https://download.docker.com/linux/ubuntu noble InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2404/x86_64  InRelease
Get:4 https://nvidia.github.io/libnvidia-container/stable/deb/amd64  Packages [18.6 kB]
Get:5 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease [256 kB]
Hit:6 http://security.ubuntu.com/ubuntu noble-security InRelease
Get:7 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease [126 kB]
Get:8 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports InRelease [126 kB]
Get:9 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates/main amd64 Packages [1067 kB]
Get:10 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates/main amd64 Components [161 kB]
Get:11 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates/universe amd64 Packages [1063 kB]
Get:12 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates/

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 5849 kB in 1s (6463 kB/s)
Selecting previously unselected package libnvidia-container1:amd64.
(Reading database ... 113813 files and directories currently installed.)
Preparing to unpack .../libnvidia-container1_1.17.6-1_amd64.deb ...
Unpacking libnvidia-container1:amd64 (1.17.6-1) ...
Selecting previously unselected package libnvidia-container-tools.
Preparing to unpack .../libnvidia-container-tools_1.17.6-1_amd64.deb ...
Unpacking libnvidia-container-tools (1.17.6-1) ...
Selecting previously unselected package nvidia-container-toolkit-base.
Preparing to unpack .../nvidia-container-toolkit-base_1.17.6-1_amd64.deb ...
Unpacking nvidia-container-toolkit-base (1.17.6-1) ...
Selecting previously unselected package nvidia-container-toolkit.
Preparing to unpack .../nvidia-container-toolkit_1.17.6-1_amd64.deb ...
Unpacking nvidia-container-toolkit (1.17.6-1) ...
Setting up nvidia-container-toolkit-base (1.17.6-1) ...
Setting up libnvidia-container1:amd64 (1.17.6-1) ...
Setting up lib

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

Running kernel seems to be up-to-date.

The processor microcode seems to be up-to-date.

No services need to be restarted.

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.
time="2025-05-09T03:29:31Z" level=info msg="Config file does not exist; using empty config"
time="2025-05-09T03:29:31Z" level=info msg="Wrote updated config to /etc/docker/daemon.json"
time="2025-05-09T03:29:31Z" level=info msg="It is recommended that docker daemon be restarted."


<Result cmd='sudo systemctl restart docker' exited=0>

In [111]:
# s.execute("sudo apt update")

In [113]:
s.execute("nvidia-smi")

Fri May  9 03:29:47 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.05              Driver Version: 560.35.05      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100 80GB PCIe          Off |   00000000:27:00.0 Off |                    0 |
| N/A   31C    P0             41W /  300W |       1MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

<Result cmd='nvidia-smi' exited=0>

In [ ]:
# s.execute("docker run --rm --gpus all ubuntu nvidia-smi")

In the terminal, after ssh to the floating ip, run the below command in docker directory to build docker image 

In [ ]:
docker build -t jupyter-onnx -f Dockerfile.jupyter-onnx-gpu .

Now, go to persistent_data_access to mount the object stores and run docker